## Lab 6 - Fitting and validating ML models

In [1]:
import polars as pl
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import math

In [22]:
numerical_catgeorical_water_quality = pl.read_csv("./data/MinneMUDAC_raw_files/numerical_categorical_water_quality.csv")

In [25]:
drop = (
    np.intersect1d(
        ["LAKE_NAME_right", "Phosphorus_mean_right", "Secchi_mean_right"],
        numerical_catgeorical_water_quality.columns
    ).tolist()   
)

numerical_catgeorical_water_quality = (
    numerical_catgeorical_water_quality.drop(drop)
)

## Splitting into Train/Test/Validation

In [36]:
# split into unique lakes
unique_lakes = (
    numerical_catgeorical_water_quality
        .select("LAKE_NAME")
        .unique()
        .with_row_count("row_id")
)
# randomly shuffle and assign splits
unique_lakes = unique_lakes.sample(fraction=1.0, with_replacement=False, seed=42)
# compute split sizes
n = unique_lakes.height
n_val = int(0.30 * n)      # 30%
n_train = int(0.50 * n)    # 50%
# remainder goes to test
# assign labels
lake_splits = (
    unique_lakes
        .with_columns(
            pl.when(pl.col("row_id") < n_val)
              .then(pl.lit("Validation"))
            .when(pl.col("row_id") < n_val + n_train)
              .then(pl.lit("Training"))
            .otherwise(pl.lit("Test"))
            .alias("set")
        )
        .select("LAKE_NAME", "set")
)
numerical_catgeorical_water_quality_split = numerical_catgeorical_water_quality.join(
    lake_splits,
    on="LAKE_NAME",
    how="left"
)

C:\Users\fx3734qp\AppData\Local\Temp\ipykernel_7492\2238772482.py:6: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("row_id")


## Phosphorus Mean CART and Random Forest Grid Search

In [43]:
TARGET_VAR = "Phosphorus_mean"
FEATURES = [
    'mean_acres', 'median_acres', 'mean_sqft', 'median_sqft', 'mean_EMV_TOTAL', 
    'median_EMV_TOTAL', 'mean_total_tax', 'median_total_tax', 'count_GARAGE_Y', 
    'total_parcels', 'count_HOMESTEAD_Y', 'count_DWELL_TYPE_CONDO', 
    'count_DWELL_TYPE_SINGLE_FAMILY', 'count_DWELL_TYPE_MISSING', 'pct_GARAGE_Y', 
    'pct_HOMESTEAD_Y', 'pct_DWELL_TYPE_CONDO', 'pct_DWELL_TYPE_SINGLE_FAMILY', 
    'pct_DWELL_TYPE_MISSING'
]

train_df_polars = numerical_catgeorical_water_quality_split.filter(pl.col("set") == "Training")

X_train = train_df_polars.select(FEATURES).to_pandas()
y_train = train_df_polars.select(pl.col(TARGET_VAR)).to_pandas().values.ravel()

test_df_polars = numerical_catgeorical_water_quality_split.filter(pl.col("set") == "Test")

X_test = test_df_polars.select(FEATURES).to_pandas()
y_test = test_df_polars.select(pl.col(TARGET_VAR)).to_pandas().values.ravel()

validation_df_polars = numerical_catgeorical_water_quality_split.filter(pl.col("set") == "Validation")

X_val = validation_df_polars.select(FEATURES).to_pandas()
y_val = validation_df_polars.select(pl.col(TARGET_VAR)).to_pandas().values.ravel()


Data for Phosphorus_mean is prepared: Train=264, Test=110, Val=154


In [45]:
print("\n--- Running Grid Search for CART ---")
cart_params = {"max_depth": [5, 10, 15], "min_samples_split": [2, 5, 10]}
cart_model = DecisionTreeRegressor(random_state=42)
cart_grid = GridSearchCV(cart_model, cart_params, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
cart_grid.fit(X_train, y_train)
best_cart_model = cart_grid.best_estimator_
print(f"Best CART Params: {cart_grid.best_params_}")

print("\n--- Running Grid Search for Random Forest ---")
rf_params = {'n_estimators': [50, 100, 200], 'max_depth': [10, 20], 'min_samples_leaf': [1, 5]}
rf_model = RandomForestRegressor(random_state=42)
rf_grid = GridSearchCV(rf_model, rf_params, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
rf_grid.fit(X_train, y_train)
best_rf_model = rf_grid.best_estimator_
print(f"Best RF Params: {rf_grid.best_params_}")


--- Running Grid Search for CART ---
Best CART Params: {'max_depth': 5, 'min_samples_split': 5}

--- Running Grid Search for Random Forest ---
Best RF Params: {'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 200}


In [46]:
cart_test_preds = best_cart_model.predict(X_test)
rf_test_preds = best_rf_model.predict(X_test)

cart_rmse = np.sqrt(mean_squared_error(y_test, cart_test_preds))
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_test_preds))

print("\n--- Test Set Comparison (RMSE) ---")
print(f"CART Test RMSE: {cart_rmse:.4f}")
print(f"Random Forest Test RMSE: {rf_rmse:.4f}")

if cart_rmse < rf_rmse:
    best_overall_model = best_cart_model
    model_type = "CART (Decision Tree)"
else:
    best_overall_model = best_rf_model
    model_type = "Random Forest"

print(f"\n] Best Overall Model: {model_type}")


--- Test Set Comparison (RMSE) ---
CART Test RMSE: 109.0490
Random Forest Test RMSE: 99.2256

🥇 Best Overall Model: Random Forest


In [47]:
train_test_df_polars = pl.concat([train_df_polars, test_df_polars])

X_refit = train_test_df_polars.select(FEATURES).to_pandas()
y_refit = train_test_df_polars.select(pl.col(TARGET_VAR)).to_pandas().values.ravel()

best_overall_model.fit(X_refit, y_refit)

val_preds = best_overall_model.predict(X_val)

final_val_rmse = np.sqrt(mean_squared_error(y_val, val_preds))
final_val_r2 = r2_score(y_val, val_preds)

print(f"Final Model Type: {model_type}")
print(f"Validation RMSE: {final_val_rmse:.4f}")
print(f"Validation R-squared (R2): {final_val_r2:.4f}")


--- Running Final Validation ---
Final Model Type: Random Forest
Validation RMSE: 56.7795
Validation R-squared (R2): -1.5718


## Secchi Mean CART and Random Forest Grid Search

In [48]:
TARGET_VAR = "Secchi_mean" 
FEATURES = [
    'mean_acres', 'median_acres', 'mean_sqft', 'median_sqft', 'mean_EMV_TOTAL', 
    'median_EMV_TOTAL', 'mean_total_tax', 'median_total_tax', 'count_GARAGE_Y', 
    'total_parcels', 'count_HOMESTEAD_Y', 'count_DWELL_TYPE_CONDO', 
    'count_DWELL_TYPE_SINGLE_FAMILY', 'count_DWELL_TYPE_MISSING', 'pct_GARAGE_Y', 
    'pct_HOMESTEAD_Y', 'pct_DWELL_TYPE_CONDO', 'pct_DWELL_TYPE_SINGLE_FAMILY', 
    'pct_DWELL_TYPE_MISSING'
]

train_df_polars = numerical_catgeorical_water_quality_split.filter(pl.col("set") == "Training")

X_train = train_df_polars.select(FEATURES).to_pandas()
y_train = train_df_polars.select(pl.col(TARGET_VAR)).to_pandas().values.ravel()

test_df_polars = numerical_catgeorical_water_quality_split.filter(pl.col("set") == "Test")

X_test = test_df_polars.select(FEATURES).to_pandas()
y_test = test_df_polars.select(pl.col(TARGET_VAR)).to_pandas().values.ravel()

validation_df_polars = numerical_catgeorical_water_quality_split.filter(pl.col("set") == "Validation")

X_val = validation_df_polars.select(FEATURES).to_pandas()
y_val = validation_df_polars.select(pl.col(TARGET_VAR)).to_pandas().values.ravel()

In [49]:
print("\n--- Running Grid Search for CART ---")
cart_params = {"max_depth": [5, 10, 15], "min_samples_split": [2, 5, 10]}
cart_model = DecisionTreeRegressor(random_state=42)
cart_grid = GridSearchCV(cart_model, cart_params, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
cart_grid.fit(X_train, y_train)
best_cart_model = cart_grid.best_estimator_
print(f"Best CART Params: {cart_grid.best_params_}")

print("\n--- Running Grid Search for Random Forest ---")
rf_params = {'n_estimators': [50, 100, 200], 'max_depth': [10, 20], 'min_samples_leaf': [1, 5]}
rf_model = RandomForestRegressor(random_state=42)
rf_grid = GridSearchCV(rf_model, rf_params, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
rf_grid.fit(X_train, y_train)
best_rf_model = rf_grid.best_estimator_
print(f"Best RF Params: {rf_grid.best_params_}")


--- Running Grid Search for CART ---
Best CART Params: {'max_depth': 15, 'min_samples_split': 5}

--- Running Grid Search for Random Forest ---
Best RF Params: {'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 200}


In [50]:
cart_test_preds = best_cart_model.predict(X_test)
rf_test_preds = best_rf_model.predict(X_test)

cart_rmse = np.sqrt(mean_squared_error(y_test, cart_test_preds))
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_test_preds))

print("\n--- Test Set Comparison (RMSE) ---")
print(f"CART Test RMSE: {cart_rmse:.4f}")
print(f"Random Forest Test RMSE: {rf_rmse:.4f}")

if cart_rmse < rf_rmse:
    best_overall_model = best_cart_model
    model_type = "CART (Decision Tree)"
else:
    best_overall_model = best_rf_model
    model_type = "Random Forest"

print(f"\n Best Overall Model: {model_type}")


--- Test Set Comparison (RMSE) ---
CART Test RMSE: 1.2086
Random Forest Test RMSE: 0.8745

🥇 Best Overall Model: Random Forest


In [51]:
print("\n--- Running Final Validation ---")
train_test_df_polars = pl.concat([train_df_polars, test_df_polars])

X_refit = train_test_df_polars.select(FEATURES).to_pandas()
y_refit = train_test_df_polars.select(pl.col(TARGET_VAR)).to_pandas().values.ravel()

best_overall_model.fit(X_refit, y_refit)

val_preds = best_overall_model.predict(X_val)

final_val_rmse = np.sqrt(mean_squared_error(y_val, val_preds))
final_val_r2 = r2_score(y_val, val_preds)

print(f"Final Model Type: {model_type}")
print(f"Validation RMSE: {final_val_rmse:.4f}")
print(f"Validation R-squared (R2): {final_val_r2:.4f}")


--- Running Final Validation ---
Final Model Type: Random Forest
Validation RMSE: 1.1730
Validation R-squared (R2): -0.6461


## Summary

The comprehensive analysis aimed to predict two distinct water quality targets using parcel features: Phosphorus Mean and Secchi Mean. For both targets, the Random Forest Regressor proved to be the superior model, consistently showing a lower Root Mean Squared Error (RMSE) on the Test Set compared to the CART model. For the Phosphorus Mean, the Random Forest achieved a Test RMSE of 99.23, while the Secchi Mean achieved a Test RMSE of 0.8745. However, the final performance evaluation on the completely isolated Validation Set revealed a critical issue for both targets, yielding a negative $\text{R}^2$ ($-1.5718$ for the Phosphorus Mean and $-0.6461$ for the Secchi Mean). A negative $\text{R}^2$ indicates that the final, refitted models performed worse than simply predicting the mean of the target variable for the lakes in the Validation Set. This strongly suggests that the lake level split failed to create representative groups, and the $\text{30\%}$ Validation subset contains lakes whose characteristics and relationships are fundamentally different from the lakes used for training the model. The models failed to generalize to these specific validation lakes, requiring further inspection of the lake data distribution.